In [116]:
# import dependencies
import pymongo
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
# import time
from selenium import webdriver

In [70]:
# connections to mongo db saving
#  !!! need to run mongo in terminal  !!!!!
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)   # Define the 'classDB' database in Mongo

# Here, db.students refers to the collection 'classroom '
db = client.mars_db   # create mars database assign to db if doesn't exist, make connection
collection = db.mars_coll  # assign collection to collection called articles,  called articles

In [71]:
### 1. NASA Mars News
# Windows Chrome Browser for use with Splinter/ webpage does not load properly without
executable_path = {'executable_path': 'chromedriver.exe'}  # will see a new chrome browser
browser = Browser('chrome', **executable_path, headless=False)


In [72]:
time.sleep(2)
url='https://mars.nasa.gov/news/'
# Retrieve page with the requests module
browser.visit(url)
response = browser.html
time.sleep(2)

In [73]:
# loup through boutiful soup results, parse artiles, make dictionary and enter into database
soup = BeautifulSoup(response, 'html.parser')
mars_news = soup.find('div', class_= "list_text")  #  class_="content_title")
mars_news = mars_news.a.text
news_p = soup.find('div', class_="article_teaser_body")
news_p = news_p.text

mars_dict = {'title':mars_news,
               'paragraph':news_p}
print(mars_dict)

{'title': 'MOXIE Could Help Future Rockets Launch Off Mars', 'paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."}


In [74]:
### 2. JPL Mars Space Images - Featured Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# use Splinter  * Make sure to find the image url to the full size `.jpg` image.
#    see if this works without changing the browser

browser.visit(url)
response = browser.html  # tell chtrome browser to visit pageExteract curretn html from page 

soup = BeautifulSoup(response, 'html.parser')
results = soup.find_all('a', class_="fancybox")
# nov 19 was /spaceimages/images/largesize/PIA24247_hires.jpg


In [75]:
i = 0
for result in results:
    if i == 1:
        featured_image_url = result['data-fancybox-href']
        break
    i = i +1
featured_image_url = f'https://www.jpl.nasa.gov{featured_image_url}'
mars_dict.update({"featured_image_url" : featured_image_url})
featured_image_url
#'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24247_hires.jpg'

In [76]:
### 3. Mars Facts
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url, index_col=0)
tables
specs = tables[0]

specs = specs.rename(columns={1:'Mars'})
specs.index.names = ['Description']

# * Use Pandas to convert the data to a HTML table string.
mars_spec_table = specs.to_html()
mars_dict.update({"Mars_Table" : mars_spec_table})
# specs_html = specs.to_html("Mars_specs.html", index=False)
# print(specs_html)
specs

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [77]:
mars_dict

{'title': 'MOXIE Could Help Future Rockets Launch Off Mars',
 'paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24247_hires.jpg',
 'Mars_Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)<

In [135]:
### 4. Mars Hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
response = browser.html

In [134]:
# find the titles on the 4 mars hemispheres that will be used
# for clicking and also the titles for the image in the dictionary.
soup = BeautifulSoup(response, 'html.parser')
titles = soup.find_all('h3')
title_list = []
for title in titles:
    title_list.append(title.text)
    print(title.text)
print(title_list)

[]


In [126]:
# url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# browser.visit(url)
# response = browser.html


# for title in title_list:
#     browser.links.find_by_partial_text(title).click()
    
#     print(title)

In [132]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
response = browser.html
hemisphere_image_urls = {}

for title in title_list:
    browser.links.find_by_partial_text(title).click()
    response = browser.html
    soup = BeautifulSoup(response, 'html.parser')

    results = soup.find('li')  
    
#     print(results.a['href'])
    imag_url = results.a['href']
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    response = browser.html
        # create dictionayr of images
    hem_dic= {"title":title,"img_url":imag_url}
    hemisphere_image_urls[title] = hem_dic
    
# mars_dict.update({"Hem_list" : hemisphere_image_urls})

ElementDoesNotExist: no elements could be found with link by partial text "Cerberus Hemisphere Enhanced"

In [131]:
hemisphere_image_urls

{}

In [67]:
# url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# response = requests.get(url)
hemisphere_image_urls = []

results = browser.find_by_css("a.product-item h3")
soup = BeautifulSoup(browser.html, 'html.parser')

    
#soup = BeautifulSoup(response, 'html.parser')
results = soup.find_all('div', class_='product-item')

for i in range(len(results)):
    
    browser.find_by_css("a.product-item h3")[i].click() #  click on link to go to image
    sampletext = browser.links.find_by_text('Sample').first
    print("test",i, sampletext)
    #print(sampletext)
    find_elements_by_tag_name('a')[0].click()
    # href="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"
    # src="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"
    

      
    # create dictionayr of images
    #hem_dic= {"title":title,"img_url":'test'}
    #hemisphere_image_urls.append

#     Browser.click(result)  # this should be a <splinter.elementList at 0x7f...

mars_dict.update({"Hem_list" : hemisphere_image_urls})

results

[]

In [65]:
for result in results:
    print(result)

In [47]:
url= 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

results = soup.find('li')  
print(results)
print(results.a['href'])
# imag_url = results.a['href']
# imag_url    

<li><a href="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg


In [ ]:
### 4. Mars Hemispheres
# url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# response = requests.get(url)  


# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# * Use Pandas to convert the data to a HTML table string.
#                                                                browser.link.find_by_partial_text('next').click()

# img_url = 
# title = 
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]


In [ ]:
results
# href="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"
# src="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"
print(results.a['href'])

In [37]:
# collection.drop()    #  .remove()
# collection.insert(mars_dict)   # insert_one of insert_many([{}.{}]) first aregument is the filer
collection.update({}, mars_dict, upsert=True)  # update database
mars_coll = collection.find()   # db.collection.find()    read database
# mars_coll = collection.find(continent: "Africa")


C:\Users\paule\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
browser.quit()


{'title': 'MOXIE Could Help Future Rockets Launch Off Mars', 'paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24247_hires.jpg', 'Mars_Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td

In [39]:
print(mars_dict)

{'title': 'MOXIE Could Help Future Rockets Launch Off Mars', 'paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24247_hires.jpg', 'Mars_Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td